In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
from common.src.main.python.utils.hdfs_generic import *
import os

MAX_N_EXECUTORS=15
MIN_N_EXECUTORS=1
N_CORES_EXECUTOR=4
EXECUTOR_IDLE_MAX_TIME=120
EXECUTOR_MEMORY='32g'
DRIVER_MEMORY='16g'
N_CORES_DRIVER=1
MEMORY_OVERHEAD=N_CORES_EXECUTOR*2048
#QUEUE="root.datascience.normal"
QUEUE="root.BDPtenants.es.medium"

BDA_CORE_VERSION="1.0.0"

SPARK_COMMON_OPTS=os.environ.get('SPARK_COMMON_OPTS', '')
SPARK_COMMON_OPTS+=" --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
SPARK_COMMON_OPTS+=" --conf spark.shuffle.manager=tungsten-sort"
SPARK_COMMON_OPTS+="  --queue %s" % QUEUE
APP_NAME='estadisticas_operadores'

# Dynamic allocation configuration
SPARK_COMMON_OPTS+=" --conf spark.driver.allowMultipleContexts=true"
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.enabled=true"
SPARK_COMMON_OPTS+=" --conf spark.shuffle.service.enabled=true"
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
SPARK_COMMON_OPTS+=" --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
SPARK_COMMON_OPTS+=" --conf spark.ui.port=58201"
SPARK_COMMON_OPTS+=" --conf spark.port.maxRetries=200"
SPARK_COMMON_OPTS+=" --executor-cores=%s" % (N_CORES_EXECUTOR)
SPARK_COMMON_OPTS+=" --conf spark.app.name=%s" % (APP_NAME)

BDA_ENV = os.environ.get('BDA_USER_HOME', '')

# Attach bda-core-ra codebase
SPARK_COMMON_OPTS+=" --files \
{}/scripts/properties/red_agent/nodes.properties,\
{}/scripts/properties/red_agent/nodes-de.properties,\
{}/scripts/properties/red_agent/nodes-es.properties,\
{}/scripts/properties/red_agent/nodes-ie.properties,\
{}/scripts/properties/red_agent/nodes-it.properties,\
{}/scripts/properties/red_agent/nodes-pt.properties,\
{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

#print os.environ.get('SPARK_COMMON_OPTS', '')
#print os.environ.get('PYSPARK_SUBMIT_ARGS', '')

sc, sparkSession, sqlContext = run_sc()
print sc.defaultParallelism

4


In [9]:
# This literal_eval is needed since 
# we have to read from a textfile
# which is formatted as python objects.
# It is totally safe.
from ast import literal_eval

# Standard Library stuff:
from functools import partial
from datetime import date, timedelta, datetime

# Numpy stuff
from numpy import (nan as np_nan, round as np_round, int64 as np_int64)
import numpy as np


# Spark stuff
from pyspark.sql import SparkSession
from pyspark import StorageLevel
from pyspark.sql.functions import (udf, col, decode, when, lit, lower, upper, concat,
                                   translate, count, sum as sql_sum, max as sql_max, min as sql_min,
                                   round, 
                                   mean, stddev, datediff,
                                   length,
                                   countDistinct,
                                   hour, date_format, collect_set, collect_list,
                                   year, month, dayofmonth,
                                   rank, expr, lag, coalesce, row_number,
                                   isnull, isnan,
                                   unix_timestamp,
                                   regexp_replace
                                  )

from pyspark.sql.types import DoubleType, StringType, IntegerType, ArrayType, FloatType, StructType, StructField

from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline

from pyspark.ml import Pipeline

from pyspark.sql.functions import row_number, col

from pyspark.sql import DataFrameStatFunctions as statFunc

from pyspark.sql.window import Window

import json
from collections import OrderedDict

from subprocess import Popen, PIPE
import datetime, calendar
from pyspark.sql import functions as F
import datetime as dt

from pyspark.ml.feature import StandardScaler

In [10]:
import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import time
# Spark utils
from pyspark.sql.functions import (udf, col, decode, when, lit, lower, concat,
                                   translate, count, max, avg, min as sql_min,
                                   greatest,
                                   least,
                                   isnull,
                                   isnan,
                                   struct,
                                   substring,
                                   size,
                                   length,
                                   year,
                                   month,
                                   dayofmonth,
                                   unix_timestamp,
                                   date_format,
                                   from_unixtime,
                                   datediff,
                                   to_date,
                                   desc,
                                   asc,
                                   countDistinct,
                                   row_number,
                                   regexp_replace,
                                   lpad,
                                   rpad,
                                   trim,
                                   split,
                                   coalesce,
                                   array)
from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
# from pyspark.ml import Pipeline
# from pyspark.ml.classification import RandomForestClassifier
# from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import datetime as dt

In [11]:
spark = (SparkSession.builder
         .master("yarn")
         .config("spark.submit.deployMode", "client")
         .config("spark.ui.showConsoleProgress", "true")
         .enableHiveSupport()
         .getOrCreate()
         )

# sc = spark.sparkContext

In [12]:
%load_ext autoreload
%autoreload 2

#import re
import subprocess
#import sys
import time

from IPython.display import HTML, display
import tabulate

def printHTML(df, sample=7):
    display(HTML(tabulate.tabulate([df.columns]+df.take(sample), tablefmt='html', headers='firstrow')))
    
# Spark utils
from pyspark.sql.functions import (array_contains, bround, col, collect_set, concat, count, decode, desc, 
                                   isnull, length, lit, lower, lpad, max as sql_max, 
                                   size, struct, substring, sum as sql_sum, 
                                   translate, trim, udf, upper, when
                                  )
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructField, StructType

import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Importo el dataset de test con las variables básicas 

In [18]:
test_basic = spark.read.load('/data/udf/vf_es/churn/portabPropension_model/test_final_30sept')

In [19]:
test_basic=test_basic.select('Cust_Agg_L2_fbb_fx_first_days_since_nc','Cust_Agg_L2_fixed_fx_first_days_since_nc','Cust_Agg_L2_mobile_fx_first_days_since_nc',
'Cust_Agg_L2_tv_fx_first_days_since_nc','Cust_Agg_fbb_services_nc','Cust_Agg_fixed_services_nc','Cust_Agg_mobile_services_nc',
'Cust_Agg_tv_services_nc', 'Bill_N1_Amount_To_Pay','GNV_Data_L2_total_data_volume','Operador_target')

In [28]:
#Para las variables del grupo 1, antes de calcular la media elimino los nulos (los -1)


grupo1=['Cust_Agg_L2_fbb_fx_first_days_since_nc','Cust_Agg_L2_fixed_fx_first_days_since_nc','Cust_Agg_L2_mobile_fx_first_days_since_nc',
'Cust_Agg_L2_tv_fx_first_days_since_nc','GNV_Data_L2_total_data_volume'] 

for variable in grupo1[0:len(grupo1)]:
        
    if variable==grupo1[0]:
        
        test_filtered=test_basic.filter((test_basic[variable]!=-1))
        mean1=test_filtered.groupby('Operador_target').mean(variable)  #con la primera variable creo el dataframe, luego hago inner joins
        
    else:

        test_filtered=test_basic.filter((test_basic[variable]!=-1))

        mean=test_filtered.groupby('Operador_target').mean(variable)

        mean1=mean1.join(mean,on='Operador_target',how='inner')

In [29]:
printHTML(mean1)

Operador_target,avg(Cust_Agg_L2_fbb_fx_first_days_since_nc),avg(Cust_Agg_L2_fixed_fx_first_days_since_nc),avg(Cust_Agg_L2_mobile_fx_first_days_since_nc),avg(Cust_Agg_L2_tv_fx_first_days_since_nc),avg(GNV_Data_L2_total_data_volume)
1,586.105,1206.55,1708.91,479.749,6070.59
3,547.78,1091.79,1536.05,477.405,5636.82
4,620.066,1191.72,1831.2,527.303,5309.04
2,614.806,1314.95,1765.91,555.084,5224.3


In [22]:
#Para el grupo 2, no tengo que eliminar registros (se imputan con 0 o la media, por lo que no se pueden detectar los nulos)

means_grupo2=test_basic.groupby('Operador_target').mean('Cust_Agg_fbb_services_nc','Cust_Agg_fixed_services_nc','Cust_Agg_mobile_services_nc',
'Cust_Agg_tv_services_nc', 'Bill_N1_Amount_To_Pay','GNV_Data_L2_total_data_volume')

In [23]:
printHTML(means_grupo2)

Operador_target,avg(Cust_Agg_fbb_services_nc),avg(Cust_Agg_fixed_services_nc),avg(Cust_Agg_mobile_services_nc),avg(Cust_Agg_tv_services_nc),avg(Bill_N1_Amount_To_Pay),avg(GNV_Data_L2_total_data_volume)
1,0.663499,0.699715,2.11619,0.312149,76.9303,6070.59
3,0.694755,0.72945,1.99355,0.345613,77.2682,5636.82
4,0.603307,0.645988,1.94774,0.26363,69.2233,5309.04
2,0.714944,0.746474,2.16508,0.463184,82.2189,5224.3
